In [1]:
%load_ext autoreload
%autoreload 2

# Dataset Creation

In [2]:
from preprocess import make_dataset
import pandas as pd
import numpy as np
import os

In [3]:
# Extract all the midi files 

midi_files = []

for path, subdirs, files in os.walk(r'./data/'):
    for name in files:
        if (name.endswith('.mid')):
            midi_files.append(os.path.join(path, name))

In [20]:
TEST_FILE = r'Ode to Joy from the 9th Symphony.mid'
df = make_dataset(midi_files[:15], file_name="midi-dataset-mini.csv", verbose=True)


" ./data/.38 Special\Caught Up In You.mid "
Tracks:  9
" ./data/.38 Special\Fantasy Girl.mid "
Tracks:  7
" ./data/10,000 Maniacs\A Campfire Song.mid "
Tracks:  8
" ./data/101 Strings\Theme From The Godfather.mid "
Tracks:  10
" ./data/10cc\Dreadlock Holiday.1.mid "
Tracks:  1
" ./data/10cc\Dreadlock Holiday.2.mid "
Tracks:  12
" ./data/10cc\Dreadlock Holiday.3.mid "
Tracks:  19
" ./data/10cc\Dreadlock Holiday.4.mid "
Skipping corrupted file...
" ./data/10cc\Dreadlock Holiday.mid "
Tracks:  11
" ./data/10cc\I'm Not In Love.1.mid "
Tracks:  1
" ./data/10cc\I'm Not In Love.2.mid "
Tracks:  14
" ./data/10cc\I'm Not In Love.3.mid "
Tracks:  11
" ./data/10cc\I'm Not In Love.mid "
Tracks:  1
" ./data/10cc\The Things We Do for Love.mid "
Tracks:  9
" ./data/1910 Fruitgum Company\Simon Says.1.mid "
Tracks:  12


In [19]:
print(df.iloc[0])

name                           .38 Special\Caught Up In You.mid
notes         [68, 128, 129, 66, 128, 129, 68, 128, 129, 64,...
durations     [0.33333333 0.         0.16666667 ... 0.25    ...
velocities                              [79  0  0 ... 79  0  0]
Name: 0, dtype: object


In [3]:
print(len(np.frombuffer(df.iloc[0]["notes"], dtype=np.int32)))

NameError: name 'df' is not defined

In [7]:
df = pd.read_csv("midi-dataset-mini.csv")

# Dataset Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("midi-dataset.csv")

In [ ]:
df['notes'] = df.notes.apply(lambda x: [int(y) for y in str(x).removeprefix('[').removesuffix(']').split(' ') if y.isnumeric()])
df['velocities'] = df.velocities.apply(lambda x: [int(y) for y in str(x).removeprefix('[').removesuffix(']').split(' ') if y.isnumeric()])
df['times'] = df.times.apply(lambda x: [float(y) for y in str(x).removeprefix('[').removesuffix(']').split(' ') if y.replace('.', '').replace('e+', '').replace('e-','').isnumeric()])
df['durations'] = df.durations.apply(lambda x: [float(y) for y in str(x).removeprefix('[').removesuffix(']').split(' ') if y.replace('.', '').replace('e+', '').replace('e-','').isnumeric()])

In [ ]:
print(df.iloc[11088])

plt.plot(df['times'][11088])

In [ ]:
note_bins = [0 for i in range(0, 200)]
velocity_bins = [0 for i in range(0, 128)]
duration_means = []
duration_std = []

time_means = []
time_std = []

for i in range(0, len(df['notes'])):
    for x in df['notes'][i]:
        note_bins[x] += 1

    for x in df['velocities'][i]:
        velocity_bins[x] += 1
    
    duration_means.append(np.mean(df['durations'][i]))
    duration_std.append(np.std(df['durations'][i]))

    time_means.append(np.mean(df['times'][i]))
    time_std.append(np.std(df['times'][i]))


In [ ]:
plt.plot(note_bins)
plt.xlim((0, 127))
plt.ylim(0, max(note_bins[0:128]))
plt.show()

In [ ]:
p = duration_means
q = duration_std
p[np.argmax(duration_means)] = 0
q[np.argmax(duration_std)] = 0

plt.scatter(p, q)

In [ ]:
p = time_means
q = time_std
p[np.argmax(time_means)] = 0
p[np.argmax(time_means)] = 0

q[np.argmax(time_std)] = 0
q[np.argmax(time_std)] = 0

plt.scatter(p,q)

In [ ]:
plt.plot(velocity_bins)
plt.xlim((0, 127))
plt.ylim(0, max(velocity_bins[0:128]))
plt.show()

In [ ]:
# Analysis specific to duration.
# We want to determine if we can quantize it some Gaussian noise

vals = []
losses_mean = []
losses_std = []
D = 32

for D in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]:
    L = []
    for track in df['durations']:
        p = np.array(track)
        l = D * p - np.trunc(D * p) 

        L.append(np.mean(l))
    L = np.array(L)
    losses_mean.append(np.mean(L))
    losses_std.append(np.std(L))

        

In [ ]:
plt.plot(losses_mean)
plt.plot(losses_std)